In [31]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [32]:
using LinearAlgebra, Optim
using BenchmarkTools, Distributions, CSV, DataFrames

In [42]:
ω_c = 5
ω_q = 2
χ = 3e-3
N = 8
amp_c = 1
amp_q = 1

1

In [43]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)

#X_gate = kron(x_gate(N, Array[[2,3], [4,5]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/4), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

ψ_initial = zeros(N)
ψ_initial[1] = 1
ψ_initial = kron(ψ_initial/norm(ψ_initial), [0,1])

ψ_initial_rand = rand(N)
ψ_initial_rand = kron(ψ_initial_rand/norm(ψ_initial_rand), [0,1])

16-element Vector{Float64}:
 0.0
 0.4532718747813395
 0.0
 0.2754868443968433
 0.0
 0.2162821383713512
 0.0
 0.32359073305380204
 0.0
 0.02427659260659347
 0.0
 0.4864281619445819
 0.0
 0.35896796806917347
 0.0
 0.4484450561870605

In [44]:
ψ_final_u = Udue * ψ_initial_rand
ψ_final_x = RX * ψ_initial_rand
ψ_final_x_gate = X_gate * ψ_initial_rand
ψ_final_superposition = kron(ones(N)/norm(ones(N)), [0,1])

ψ_final_k = zeros(N)
ψ_final_k[4] = 1
ψ_final_k = kron(ψ_final_k/norm(ψ_final_k), [0,1])

16-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [45]:
# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0], [0 0 … 0 0; 1 0 … 0 0; … ; 0 0 … 0 0; 0 0 … 1 0], [1 0 … 0 0; 0 -1 … 0 0; … ; 0 0 … 1 0; 0 0 … 0 -1])

In [46]:
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

snap (generic function with 1 method)

In [47]:
function cost(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function cost_ecd_rot_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ECD(β_i[i] + β_r[i] * 1im) * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)'
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end

function gradient(coefficients, unitary, N, h, cost_before_increment)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost(coeffs_copy, unitary, N) - cost_before_increment) / h
    end

    return gradient

end

function gate_composition(coefficients, N, l_coeff)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return propagator
    
end

function cost_state_prep(coefficients, N, ψ_n, ψ_m)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(ψ_m' * propagator * ψ_n)^2
    
end


function gradient_state_prep(coefficients, N, h, cost_before_increment, ψ_n, ψ_m)

    L = length(coefficients)
    gradient = zeros(L)

    for i in 1:L
        coeffs_copy = copy(coefficients)
        coeffs_copy[i] = coefficients[i] + h
        gradient[i] = (cost_state_prep(coeffs_copy, N, ψ_n, ψ_m) - cost_before_increment) / h
    end

    return gradient

end

gradient_state_prep (generic function with 1 method)

In [48]:
snapgate = snap(rand(N))

16×16 Matrix{ComplexF64}:
 0.999786-0.0206763im       0.0-0.0im        …       0.0+0.0im
      0.0-0.0im        0.999786-0.0206763im          0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im        …       0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im        …       0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0+0.0im
      0.0+0.0im             0.0+0.0im                0.0-0.0im
      0.0+0.0im             0

## STATE PREP

In [49]:
x(x) = cost_state_prep(x, N * 2, ψ_initial_rand, ψ_final_u)
rx(x) = cost_state_prep(x, N * 2, ψ_initial_rand, ψ_final_x)
u(x) = cost_state_prep(x, N * 2, ψ_initial_rand, ψ_final_x_gate)
k(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_k)
superposition(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_superposition)
it_costs_x = []
coefficients_x = []
it_costs_rx = []
coefficients_rx = []
it_costs_u = []
coefficients_u = []
it_costs_superposition = []
coefficients_superposition = []
it_costs_k = []
coefficients_k = []
blocks = []
for depth in 1:1:10
    for iteration in 1:5
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res_x = Optim.optimize(x, coefficients , LBFGS(), Optim.Options())
        res_rx = Optim.optimize(rx, coefficients , LBFGS(), Optim.Options())
        res_u = Optim.optimize(u, coefficients , LBFGS(), Optim.Options())
        res_superposition = Optim.optimize(superposition, coefficients , LBFGS(), Optim.Options())
        res_k = Optim.optimize(k, coefficients , LBFGS(), Optim.Options())
        push!(blocks, depth)
        push!(it_costs_x,Optim.minimum(res_x))
        push!(it_costs_rx,Optim.minimum(res_rx))
        push!(it_costs_u,Optim.minimum(res_u))
        push!(it_costs_k,Optim.minimum(res_k))
        push!(it_costs_superposition,Optim.minimum(res_superposition))
        push!(coefficients_x,Optim.minimizer(res_x))
        push!(coefficients_rx,Optim.minimizer(res_rx))
        push!(coefficients_u,Optim.minimizer(res_u))
        push!(coefficients_k,Optim.minimizer(res_k))
        push!(coefficients_superposition,Optim.minimizer(res_superposition))
    end
end

blocks: 1 iterations :1
blocks: 

1 iterations :2


blocks: 1 iterations :3
blocks: 

1 iterations :4


blocks: 1 iterations :5
blocks: 

...

blocks: 3 iterations :1
blocks: 

3 iterations :2


blocks: 3 iterations :3
blocks: 

3 iterations :4


blocks: 3 iterations :5
blocks: 

In [ ]:
df_x = DataFrame(fidelity=it_costs_x, blocks=blocks, coefficients=coefficients_x)
df_rx = DataFrame(fidelity=it_costs_rx, blocks=blocks, coefficients=coefficients_rx)
df_u = DataFrame(fidelity=it_costs_u, blocks=blocks, coefficients=coefficients_u)
df_k = DataFrame(fidelity=it_costs_k, blocks=blocks, coefficients=coefficients_k)
df_superposition = DataFrame(fidelity=it_costs_superposition, blocks=blocks, coefficients=coefficients_superposition)
CSV.write("../../data/state_prep/7_photons/ECD/x_gate.txt", df_x)
CSV.write("../../data/state_prep/7_photons/ECD/rx_gate.txt", df_rx)
CSV.write("../../data/state_prep/7_photons/ECD/u_gate.txt", df_u)
CSV.write("../../data/state_prep/7_photons/ECD/k_gate.txt", df_k)
CSV.write("../../data/state_prep/7_photons/ECD/superposition_gate.txt", df_superposition)

## GATE PREP

In [9]:
f(x) = cost(x, RX, N * 2)
h(x) = cost(x, X_gate, N*2)
g(x) = cost_state_prep(x, N * 2, ψ_initial, ψ_final_x_gate)

g (generic function with 1 method)

In [13]:
costs = []
for depth in 36:6:36
    it_costs = []
    for iteration in 1:3
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # coefficients = [0.6607403361268763, 0.08590035700013822, -0.15387625899517954, -0.0960644983553175, 0.9153959795248622, 0.007724187879161347, -0.5090330979766927, 0.9848953390956963, -1.0253742246379498, -0.9079754831003415, 0.4169409747817356, 0.610818000523414, 0.07128328615123676, 0.403077332452158, -0.6109122881484295, -1.0360971047664385, 1.1262766239260216, -0.6304249625105642, -0.2351275138303528, -0.11113963107255552, -0.30727857275776893, -0.9560156436783254, -0.9588641618503907, -0.5615110400608102, 0.6979359775990401, -1.3479319230828708, 1.144999968863136, -0.02452555473365016, 0.28932229947357263, -0.9045127785127591, 1.8765908283107466, -0.736873707459418, -0.28850586435606757, 0.5884733738934329, 1.4508285269783043, 0.8693956490351282, 0.5201973812599108, -0.685378250131944, -1.0886541906982647, 1.8693999741905731, 0.6165048375249106, -0.09818611839899977, 0.46873629214852514, 0.4880095274245698, 0.38106149957747726, -0.5827889973241523, -0.6905369571222512, 1.1658065887145483, 1.062568599032343, -0.18504291660841607, 0.686436753332916, -0.20247952401223734, 1.3550185214359076, -1.049420992328533, -0.5632420836010926, 1.7038757751688596, 0.3226558816959283, 0.3754328410298876, 0.3422996942070637, -1.9216904514476205, 0.765467809470165, -0.625328933952218, 0.460858987914061, 0.002075990022226634, 1.2798518186476275, 0.5792481155683049, 1.8089242646408938, -0.14210963784547428, 0.273376746253454, 0.3953776936981302, 0.059444566807252006, -1.6473522057488557, 0.40546454625375183, 1.334454823223599, -2.0521180040535296, -1.446196714956497, -1.665926876000584, 2.254955340148629, -2.297956375328917, -2.270122132232124, -1.3449304232202812, 2.393658727122352, -1.3937699546537496, -1.9455685308128177, 2.3894922301485493, -4.335282724063245, -2.3542075066493013, 1.9143264382319582, -0.13517376829314726, 4.204753311113465, -1.0469871024147461, 1.9056865365157536, -1.7955397638510877, -1.73177972680147, -0.7509673239374065, -2.6024383137053255, 0.7419271419107999, -2.71856582176325, -1.2887694928453797, 1.8542458400886204, 2.0589773046176676, -0.9401399462050072, -2.0699461300741917, -2.4635963338325437, 1.6648474997540064, 4.656256791144964, -1.1964398954057653, 1.9259286028827043, 1.0670338513652182, 0.5350802165298844, -1.6779453492197023, 2.7659679865098314, 1.6602430426705388, -1.1023298777925417, 3.090975382255604, -0.5884862701312702, 0.1546958792130905, -1.6422006553717334, 1.4036732165290342, 1.4587548127932946, 3.1580771355302293, 2.8704969724586715, 2.7211532765011355, -4.363089774515921, 2.6576492378347636, -2.1030741116467735, -1.8108492640237228, 3.5286268023184473, -2.186231056261032, -1.5713913681549978, 1.2833389769352732, -3.2762543436035005, -1.7029746023165908, -1.8674116557548488, 2.978774508450448, -1.413300623904884, -0.18526406254899538, 1.6496378078056355, 2.4658709280349713, 4.353724774171218, -2.740403928025058, -3.0881739912095836, 1.6714155279189957, -1.4221040973530892]
        # opt_coefficients = [0.13127246520334743, -0.019497584668193676, 0.08441258787959842, 0.18853144018033427, 2.4773268321055526, -0.14068538254906235, 0.1067057733639206, 0.1349017917477391, -0.28172200750776616, 2.3343572457141644, 0.23803460000361065, 0.02224019699673719, -0.02819220072444795, 0.14025366097313285, -0.039304691135796384, 0.034017130408503904, -1.270709512762515, 1.1222318350735496, -0.5050327642586442, 0.44136685654889646, -0.11819305256693396, -0.0171689224126802, 0.09552182521389328, -0.2328982824584922, -0.18887578187905554, -0.15747221392930363, 0.048819896470870795, 0.06658477533856631, -0.22675451226200594, -0.0731322584470268, 1.0320426049171438, 0.6225620315319085, 1.568514654834306, 0.7262178776748648, -0.22834995134017863, 0.3779892846129676, 0.4262227353912679, 0.3196753483381889, 0.2953992663817206, -0.19218722459895415, 1.0202317022438367, 1.705913290128121, 0.6088975228114619, 1.3686579107788293, -0.08953083032928448, 1.3785033310248134, 0.6210712278453568, -0.546707314086662, 0.5554710674453178, -0.738159834335507, 0.8480966018787588, 1.3078085508366757, 0.8089514554005326, -1.469472380398311, 0.18028304122964178, 0.17800584503049802, -0.16671991762565194, 0.15198499049007122, -0.04092766820003216, 0.11711271102437686, 0.5188232840958972, -1.7867718434727076, 3.5223065386348553, 2.2224437866146936, -0.7912271654827151, 0.8138625856913634, 0.8251321109743369, 0.3908880448666587, -1.0196639363402769, -0.47085490582037853, 0.4599953976090268, 2.482555446317384, -1.8990959008772232, -2.347458571630002, 1.8940454195441299, 1.6098827996499168, 0.6858333434815906, 3.099926769099066, 0.26105209891049624, 3.116263068234298, 2.5252126200803366, 0.6579487830800986, -2.0398634955596138, -2.063231611559297, 0.17200838633826052, 0.21746703903761155, -0.08632133913870561, 0.10226589649918397, 0.11017725158881246, -0.14735488542926817, -1.1227719271090217, 2.528691011685489, -3.6943677736137874, 0.4459977262437821, 2.4117078989762133, 0.26322261352850806, 1.2256689177520266, -2.2308581588753373, 0.9892257825254104, 0.8281524197559402, 1.9598978630167818, -2.1148746299260353, -2.803899573953217, -2.245508040936566, -0.6193170598374295, 1.7795990385359928, -1.8558619160741494, 1.1841888451815246, 0.16682143048943704, 0.2751262646174936, -1.8060262029518621, 0.1641950718415823, -0.5043898627135203, 2.2137233254732416, -0.3804855296494221, 0.29075602987498295, -0.013080790874542322, -0.052115930912448594, 0.0071533840307411774, 0.023714329530033218]
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/4)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))..., opt_coefficients[Int(length(opt_coefficients)/4)+1:Int(length(opt_coefficients)/2)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:Int(3*length(opt_coefficients)/4)]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))..., opt_coefficients[Int(3*length(opt_coefficients)/4)+1:end]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res = Optim.optimize(f, coefficients , LBFGS(), Optim.Options(show_trace=true, show_every=50, iterations = 1000))
        push!(it_costs,Optim.minimum(res))
        println(Optim.minimum(res))
        println(Optim.minimizer(res))
    end
    println(minimum(it_costs))
    push!(costs,minimum(it_costs))
end

blocks: 36 iterations :1
Iter     Function value   Gradient norm 


     0     9.980577e-01     1.675169e-02
 * time: 0.00014209747314453125


    50     4.057285e-02     3.122966e-02
 * time: 90.55464506149292


   100     1.507052e-02     9.932765e-03
 * time: 186.75614213943481


   150     8.460625e-03     8.530731e-03
 * time: 286.65741896629333


...

   400     2.040622e-03     1.947829e-03
 * time: 768.1352109909058


   450     1.732400e-03     1.972260e-03
 * time: 864.1305019855499


   500     1.458865e-03     1.519331e-03
 * time: 959.4086420536041


   550     1.311239e-03     1.398968e-03
 * time: 1055.7436609268188


InterruptException: InterruptException:

In [62]:
it_costs_x = []
coefficients_x = []
it_costs_rx = []
coefficients_rx = []
blocks = []
for depth in 36:6:36
    for iteration in 1:10
        println("blocks: ", depth," iterations :", iteration)
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        # opt_coefficients = [0.0976423458354245, 0.0450708234683308, 0.10976747963502345, 0.08416609504894443, 2.551326070782372, -0.08964024119320309, 0.10485248324424644, 0.13969513913978635, -0.22349455300798343, 2.3735987675524037, 0.19185222571130106, 0.09817439361481875, 0.10464682496970999, 0.03769605958097719, 0.059524186293775296, 0.008378579058631288, -1.2534771254122576, 1.1903829602499862, 0.016016288408910853, -0.05483313987446702, -0.046034209736723807, -0.027533341989763437, 0.006811587319806202, -0.07449680205270068, 0.9909477420650091, 0.7105671478179529, 1.4506719329759887, 0.6584572437091945, -0.1462197067998505, 0.3890746504254293, 0.4550190349625619, 0.27485980171484087, 0.24866482812659083, -0.13476539605421262, 1.1225574211169433, 1.9743342171260378, 1.0562819850952958, 0.7320395937688988, 0.4678336590764824, 0.7663663217427629, 0.7493470855455572, -0.6802045420602124, 0.5414009020842547, -0.5236457353726556, 0.7006096048990366, 1.288999517730632, 0.33766541989560206, -1.3141536095001372, 1.2548422513237936, -1.4739003495829288, 3.612006758803497, 2.177902858706418, -0.52269040375497, 0.9564247538849191, 0.49002185275320925, 0.47233587796819054, -0.7410940755657739, -0.41486097094262636, 0.42027037104216425, 2.178242513687377, -1.526022531001938, -2.559186549835382, 2.114662728368936, 2.087936066989978, 0.7610469841037554, 3.1090098597892273, 0.4006039324836421, 3.1069649976511053, 1.8012672154816713, 0.5695265489909448, -2.77394600557009, -1.5567065154837294, -0.9443614233970088, 2.2647812845224053, -3.57309947272525, 0.5409027672256352, 2.425316199161032, 0.16790448062740732, 1.1584460466449906, -2.4993058076042263, 0.8206079200349031, 0.7528221404227818, 1.353149399622874, -2.1365245400174433, -2.372828756091764, -2.5847407449073145, -0.6423548068234998, 1.8509304528464354, -1.6388536401478402, 1.462231976003448, 0.605305856716617, 1.0233566302172725, -1.7465358497919055, 0.32976779077893226, -0.7074764414799524, 2.9589238120929138]
        # coefficients[1:Int(l_coeff / 2)] = [opt_coefficients[1:Int(length(opt_coefficients)/4)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))..., opt_coefficients[Int(length(opt_coefficients)/4)+1:Int(length(opt_coefficients)/2)]..., (zeros(Int(l_coeff / 4 - length(opt_coefficients)/4)))...]
        # coefficients[Int(l_coeff / 2) + 1:end] = [opt_coefficients[Int(length(opt_coefficients)/2)+1:Int(3*length(opt_coefficients)/4)]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))..., opt_coefficients[Int(3*length(opt_coefficients)/4)+1:end]..., zeros(Int(l_coeff / 4 - length(opt_coefficients)/4))...]
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        res_x = Optim.optimize(h, coefficients , LBFGS(), Optim.Options())
        res_rx = Optim.optimize(f, coefficients , LBFGS(), Optim.Options())
        push!(blocks, depth)
        push!(it_costs_x,Optim.minimum(res_x))
        push!(it_costs_rx,Optim.minimum(res_rx))
        push!(coefficients_x,Optim.minimizer(res_x))
        push!(coefficients_rx,Optim.minimizer(res_rx))
    end
end

blocks: 36 iterations :1
blocks: 

36 iterations :2


blocks: 36 iterations :3
blocks: 

36 iterations :4


blocks: 36 iterations :5
blocks: 

36 iterations :6


blocks: 36 iterations :7
blocks: 

36 iterations :8


blocks: 36 iterations :9
blocks: 

36 iterations :10


In [63]:
df_x = DataFrame(fidelity=it_costs_x, blocks=blocks, coefficients=coefficients_x)
df_rx = DataFrame(fidelity=it_costs_rx, blocks=blocks, coefficients=coefficients_rx)
CSV.write("../../data/x_gate.txt", df_x)
CSV.write("../../data/rx_gate.txt", df_rx)

"../../data/rx_gate.txt"

# stoc gradient descent

In [ ]:
costs_final = []
for depth in 64:8:64
    it_costs = []
    for iteration in 1:3
        l_coeff = depth * 4
        coefficients = zeros(l_coeff)
        coefficients[1:Int(l_coeff / 2)] = rand(Int(l_coeff / 2))*2 .-1
        coefficients[Int(l_coeff / 2) + 1:end] = rand(Int(l_coeff / 2)) * 2 * pi .- pi
        coefficients = [0.8690408632149678, -0.7236416401971805, -0.0908719850588177, 0.8258628776740167, 0.9056445769487957, -0.8220939210081029, 0.29601861853616424, 0.7813552229187792, 0.9837594907380149, -0.10686709645389225, -0.6656407250459772, 0.992278727387623, -0.4893461915705191, -1.093993962903741, 0.4845407107960842, -0.33857113975897773, -0.438797994554033, -0.5977365084149797, 0.20827466739249778, -0.8959109454265681, 0.5383716212207733, 1.3219169845803183, 0.8578064725920909, -0.7256810259845291, 0.37256259960242327, -1.2114316020995235, -0.282902962907945, -0.26590734759323503, 0.5548879566071089, 0.6803304120122661, -1.0549066677248888, 0.4013045273205048, 0.039560949181603355, 1.0529793828938236, -0.044290684794351255, -0.21790564460826214, 0.7641852601374161, 0.9310403877890154, -0.24519813978115026, 0.6528372052655407, -1.2521949229101696, -0.21682088705693697, 0.10911919627652335, -0.31451889968452584, 0.07428438971871276, 0.8168118972903352, -0.4306959382326172, -1.31201487771103, -0.5551956081923786, -0.24938592347753555, 0.7261072879386111, 0.3449918944429117, 1.0218022420859423, 0.9586632779953215, 0.9830015979456759, 1.1800344827793208, -0.6015729201693757, 0.703237921035128, -0.8659071933834304, 0.7405471254766851, 0.5502125342538493, -0.1279261558044507, 0.5436405284887846, 0.7561715320031175, -1.8294742583884145, 0.8274849373492085, -0.5023928927013781, -0.2106775708083146, -0.25623844263590434, -0.13700323024471564, -0.5427032401830472, 0.8481733035397572, 2.857123893083541, 0.6722772567617452, 3.3393904892971857, -2.7921664352181588, -1.4927954172559996, 0.3500341652427946, 2.178592973947075, 2.0048971146113073, -1.3350642362244973, 0.9711032282505817, 2.3816703973089663, -2.6138104747765247, 2.5134841942159674, -1.9047337097507626, 2.2260526473073146, -1.8796539188004344, 1.6517968966982153, 1.4620145034237324, 1.8366376463164558, -1.3614199467978896, -1.3067881525922014, -0.8271088962045482, -1.3523594955345797, -0.8216607978872826, -1.015386302411769, -1.4951025575413324, 2.199130944918348, -2.098541424724332, 1.8106459325712736, -2.7528680557326783, 1.434522172097638, 0.23235089810469334, -1.3427653914596476, -2.1865004319117882, -4.499311913607152, 3.5233665940145906, 0.11147763894762594, 1.7253123134995119, 0.3069539608771961, 0.7456729443236856, 0.9675539799184902, -2.0807423878078692, -0.21274877285924562, -2.4625967782327733, -0.8958201449217154, -0.16506248177386307, -0.7781344488152088, 0.5657089824636788, 0.49103015856219095, -1.893089322888448, 1.8586226474868552, 1.3367905090651682, -0.845322993521433, -2.414547922619251, -2.0653040809018646, 2.13700608386267, 0.6560460356595541, 1.2398223292845443, 2.962599533837159, 1.1325322497723143, 3.463842619105753, -3.938053691752728, -0.9569023982656585, 1.605238908260515, 0.6974820952130029, 2.5406564651431, 2.285077612710168, 0.027961409188739638, 1.0736422395589698, 2.4870006619787572, 0.692378916991639, -2.718696870022777]
        beta1 = 0.8
        beta2 = 0.999
        alpha = 0.01
        m = zeros(length(coefficients))
        v = zeros(length(coefficients))
        coeffs_stored = []
        costs = []
        cost_prev_it = cost(coefficients, RX, N*2)  #cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #

        for i in 1:1000
            grad = gradient(coefficients, RX, N * 2, 1e-8, cost_prev_it) #gradient_state_prep(coefficients, N*2, 1e-8, cost_prev_it, ψ_initial, ψ_final_x_gate) #
            for n in 1:length(coefficients)
                m[n] = beta1 * m[n] + ( 1 - beta1 ) * grad[n]
                v[n] = beta2 * v[n] + ( 1 - beta2 ) * grad[n]^2
                m_norm = m[n] / ( 1 - beta1^i )
                v_norm = v[n] / ( 1 - beta2^i )
                coefficients[n] = coefficients[n] - alpha * m_norm / (sqrt(v_norm) + 1e-8)
            end
            cost_prev_it = cost(coefficients, RX, N*2)# cost_state_prep(coefficients, N * 2, ψ_initial, ψ_final_x_gate) #
            push!(costs, cost_prev_it)

            if i % 50 == 0
                if norm(grad) < 1e-6
                    break
                end
                println(i)
                println(cost_prev_it)
            end
        end
        println("finished: d = ", depth)
        println(costs[end])
        push!(it_costs,costs[end])
        println(coefficients)
    end
    println(it_costs)
    println(minimum(it_costs))
    push!(costs_final, minimum(it_costs))
end
costs_final